#### 네이버 뉴스 SUB 페이지 크롤링
    - 한개의 SUB페이지 크롤링 한 후 함수로 재구성
    - 02_1 번에서 수집된 데이터를 이용해서 여러 SUB 페이지 크롤링

In [48]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen


In [49]:
#뉴스 SUB페이지 문서 코드 받아오기
url = "https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012576808"

headers = {"User-Agent":
          "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

result = requests.get(url,headers=headers)

In [50]:
result

<Response [200]>

In [51]:
#result.text  전송된 소스코드를 문자열로 반환
#result.content 전송된소스코드를 row text 형식으로 반환 , 앞에 b코드가 붙어있음

In [52]:
bs_obj = BeautifulSoup(result.content,'html.parser')
#기사 제목 추출

title = bs_obj.find(id='articleTitle')
#title.text
date_time = bs_obj.find('span',{"class":'t11'})
#date_time.text



### 개발자 도구의 copy selector 사용 방법
- 크롤링 하려는 컨텐츠에서 검사 - 해당소스에서 단축메뉴 -copy - copy selector
    - 붙여넣기하면 선택자를 반환해주는데, 선택된 내용이 없는 경우가 있음
        - 잘못된 선택자를 생성했을 경우 -> 선택자 재구성
        - 동적으로 생성되는 태그인 경우에는 선택자가 만들어지지만 정적으로 크롤링은 불가능
        - 네이버 뉴스의 서브페이지 좋아요 수와 기사 댓글 수는 동적 생성해서 정적으로 크롤링이 불가능함

bs_obj.select("#main_content > div.article_header > div.article_info > div > div.article_btns > div.article_btns_left > div > a > span.u_likeit_text._count.num")

In [53]:
#sub 페이지에서 수집된 데이터 반환하는 함수 작성(서브 페이지 url을 파라미터로 전달)

def get_news_sub_info(url) :
    #1. 전달된 url에 접속 후 응답객체 반환
    
    headers = {"User-Agent":
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

    result = requests.get(url, headers=headers)
    
    #2.bs4 객체 생성
    bs_obj=BeautifulSoup(result.content, "html.parser")
    
    #3.필요 데이터 추출
    # 기사 제목 추출(태그 객체)
    title = bs_obj.find(id='articleTitle')
    #기사 작성 시간 추출(태그 객체)
    date_time = bs_obj.find('span',{"class":'t11'})

    #기사 내용 
    article_content = bs_obj.find(id="articleBodyContents")
    
    
    #4. 수집데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub['기사제목'] = title.text
    dict_sub['기사 입력 시간'] = date_time.text
    dict_sub['기사 내용'] = article_content.text
    
    return dict_sub

In [54]:
## main: 함수호출 코드
headers= {"User-Agent":
         "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com/"
res = requests.get(url,headers=headers)
html = res.text
bs_obj = BeautifulSoup(html,"html.parser")

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul", {"class": "hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")


link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com"+a_tag["href"])


#link_list
for url in (link_list):
    res = get_news_sub_info(url)
    print(res['기사제목'])
    
    

WHO "백신 부스터샷 9월까지 유예해야…저소득국 접종 먼저"
문 대통령, 국가인권위원장 송두환·금융위원장 고승범 등 장·차관 인사 단행
내년도 최저임금 5.1% 오른 9,160원…1월 1일부터 적용
SK이노 배터리 법인 ‘SK배터리’ 10월 출범… 상장은 언제?
편의점서 쓰러진 고객에 심폐소생술 한 '알바생'…"생명 살렸다"


In [55]:
##get_news_sub_info 함수의 반환값에 기사 내용까지 포함되게 코드 수정
## 반환되는 결과값을 df로 구성하시오.

import pandas as pd

dict_sub = {}
dict_sub['기사제목'] = []
dict_sub['기사 입력 시간'] = []
dict_sub['기사 내용'] = []


news_sub = pd.DataFrame(dict_sub)
news_sub


headers= {"User-Agent":
         "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com/"
res = requests.get(url,headers=headers)
html = res.text
bs_obj = BeautifulSoup(html,"html.parser")

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul", {"class": "hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")


link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com"+a_tag["href"])


#link_list
for url in (link_list):
    res = get_news_sub_info(url)
    news_sub = news_sub.append(res, ignore_index=True)
    
news_sub

,기사제목,기사 입력 시간,기사 내용
0,"‘경쟁자 배제’ 혐의 구글, 다음 달 1일 공정위 전원회의서 최종 심의",2021.08.05. 오전 10:31,\n\n\n\n\n구글이 스마트기기 운영체제(OS)와 앱 마켓 시장에서 지배력을 남...
1,‘쩍벌’ 논란에 입 연 윤석열…“허벅지 살이 많아서...”,2021.08.05. 오전 12:11,\n\n\n\n\n윤석열 대선 예비후보가 지난 1일 오후 서울 여의도 하우스카페에서...
2,'갤럭시 언팩' 예고 영상 일주일만에 1억뷰... 폴더블 기대감 고조,2021.08.05. 오전 10:00,\n\n\n\n\n삼성전자 갤럭시 언팩 공식 트레일러 영상삼성전자 '갤럭시 언팩' ...
3,"기업가치 높일 효자 될까… SK이노, 배터리·석유개발 분사",2021.08.05. 오전 6:03,"\n\n\n\n\n2021년 10월 신설법인으로 출범SK이노, 두 회사 주식 100..."
4,강도 더 세진 ‘탄소중립 시나리오’…완벽한 ‘넷제로’ 담은 탄중委,2021.08.05. 오전 10:10,\n\n\n\n\n2050 탄소중립 시나리오 대통령 직속 탄소중립위원회(탄중위)가 ...


### 데이터 불러와서 df만들기

In [59]:
news_df = pd.read_csv("./crawlData/naver_news.csv")
news_df

,Unnamed: 0,section:,news_title,link_url
0,0,헤드라인,톱4 없는 국힘 '쪽방촌 봉사'…대선버스 출발부터 삐걱,https://news.naver.com/main/read.naver?mode=LS...
1,1,헤드라인,"SK이노도 배터리 독립시킨다, 10월 물적분할",https://news.naver.com/main/read.naver?mode=LS...
2,2,헤드라인,"사면초가 몰린 쿠오모, 스스로 물러날까…탄핵 시계도 '째깍'",https://news.naver.com/main/read.naver?mode=LS...
3,3,헤드라인,로톡 등 ‘플랫폼 광고’ 금지 규정 오늘부터 시행… ‘변호사법 위반’ 놓고 해석 갈려,https://news.naver.com/main/read.naver?mode=LS...
4,4,헤드라인,대구 121명 신규 확진…지난해 3월11일 131명 이후 최다,https://news.naver.com/main/read.naver?mode=LS...
5,5,정치,"[최강시사] 이준석 “안철수 ‘전범’ 운운, 정상적인 정치 지도자 간 대화 아냐”",https://news.naver.com/main/read.naver?mode=LS...
6,6,정치,"사흘간 영남 찾는 최재형, 월성 원전도 방문…윤석열, 나흘간 휴가",https://news.naver.com/main/read.naver?mode=LS...
7,7,정치,"김근식 “이재명, 음주운전 공격받자 김연경 끌어들여”",https://news.naver.com/main/read.naver?mode=LS...
8,8,정치,"이준석 ""안철수, 그냥 내가 싫은 거 아닌가?""",https://news.naver.com/main/read.naver?mode=LS...
9,9,정치,"'도리도리 점검단'…윤석열, '쩍벌' 이어 또 셀프디스",https://news.naver.com/main/read.naver?mode=LS...


In [57]:
# 빈 데이터 프레임 하나 생성
dict_sub = {}
dict_sub['기사제목'] = []
dict_sub['기사 입력 시간'] = []
dict_sub['기사 내용'] = []


news_sub_main = pd.DataFrame(dict_sub)
news_sub_main

,기사제목,기사 입력 시간,기사 내용


In [58]:
import time

link_list=news_df["link_url"]

for url in link_list :    
    res = get_news_sub_info(url)
    # 빈 df에 반환되는 dict 행으로 추가(append 함수 이용)
    news_sub_main = news_sub_main.append(res, ignore_index=True)
    time.sleep(2) #계속되는 요청으로 서버가 disconnect 시킬 수 있으므로 시간간격을 두어서 요청함
    
news_sub_main


,기사제목,기사 입력 시간,기사 내용
0,톱4 없는 국힘 '쪽방촌 봉사'…대선버스 출발부터 삐걱,2021.08.05. 오전 8:39,"\n\n\n\n\n국민의힘 대선 첫 이벤트 '봉사활동' 불참한 톱4""先 일정 있었다..."
1,"SK이노도 배터리 독립시킨다, 10월 물적분할",2021.08.05. 오전 12:03,\n\n\n\n\n분사 소식에 주가 3.75% 급락SK이노베이션(이하 SK이노)이 ...
2,"사면초가 몰린 쿠오모, 스스로 물러날까…탄핵 시계도 '째깍'",2021.08.05. 오전 12:51,\n\n\n\n\n사면초가에 몰린 앤드루 쿠오모 미국 뉴욕주지사가 스스로 물러나는 ...
3,로톡 등 ‘플랫폼 광고’ 금지 규정 오늘부터 시행… ‘변호사법 위반’ 놓고 해석 갈려,2021.08.05. 오전 8:54,\n\n\n\n\n전문직 분야별 플랫폼./그래픽=이주룡 기자로톡으로 대표되는 법률서...
4,대구 121명 신규 확진…지난해 3월11일 131명 이후 최다,2021.08.05. 오전 8:26,\n\n\n\n\n대부분이 교회·체육시설 관련…최근 1주일 일평균 55.7명 발생대...
5,"[최강시사] 이준석 “안철수 ‘전범’ 운운, 정상적인 정치 지도자 간 대화 아냐”",2021.08.05. 오전 9:30,"\n\n\n\n\n - 안철수, 비정상적인 대화 속 긁지 말고, 합당 여부 밝혀야 ..."
6,"사흘간 영남 찾는 최재형, 월성 원전도 방문…윤석열, 나흘간 휴가",2021.08.05. 오전 9:27,\n\n\n\n\n5일 고향 진해 찾고 6일 박정희 생가 방문7일 월성 원전 1호기...
7,"김근식 “이재명, 음주운전 공격받자 김연경 끌어들여”",2021.08.05. 오전 9:26,\n\n\n\n\n“보이콧 주장할 땐 언제고 올림픽 찬양”이재명 더불어민주당 대선 ...
8,"이준석 ""안철수, 그냥 내가 싫은 거 아닌가?""",2021.08.05. 오전 9:24,\n\n\n\n\n\t\n\t[이데일리 박지혜 기자] 국민의힘과 국민의당 합당 논의...
9,"'도리도리 점검단'…윤석열, '쩍벌' 이어 또 셀프디스",2021.08.05. 오전 9:22,"\n\n\n\n\n반려묘 내세우며 ""'도리도리' 고쳐가겠다""'쩍벌' 논란에는 ""허벅..."
